In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
!rm -rf ./logs/LSTM #dont run this line unless you want to delete the tensorboard logs

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
def labels(filename):
    with open(filename, 'rb') as f:
        labels = np.load(f).ravel()
    return labels

In [5]:
training_data_y = labels("../Labels-Train.npy")
test_y = labels("../Labels-Test.npy")

training_data_x = pickle.load(open("TS-Train.pkl", "rb"))
test_x = pickle.load(open("TS-Test.pkl", "rb"))

training_data_x = training_data_x.dropna(axis=1)
test_x = test_x.dropna(axis=1)

training_data_x = preprocessing.normalize(training_data_x, norm='max', axis=0)
test_x = preprocessing.normalize(test_x, norm='max', axis=0)

training_data_x = convert_3D(training_data_x)
test_x = convert_3D(test_x)

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [6]:
LSTMmodel = Sequential([
    LSTM(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    LSTM(64),
    #Dropout(0.3),
    Dense(1, activation='sigmoid')
])

LSTMmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.000003),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/LSTM/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasLSTM/training.csv',separator=",", append=False)

LSTMmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = LSTMmodel
model.save('keras-models/KerasLSTM')

Epoch 1/10
276/276 [==============================] - 21s 61ms/step - loss: 0.6686 - accuracy: 0.7732 - auc: 0.4628 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_negatives: 1999.0000 - false_positives: 0.0000e+00 - true_negatives: 6813.0000 - true_positives: 0.0000e+00 - val_loss: 0.6623 - val_accuracy: 0.7652 - val_auc: 0.4242 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 887.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 2890.0000 - val_true_positives: 0.0000e+00
Epoch 2/10
276/276 [==============================] - 15s 53ms/step - loss: 0.6539 - accuracy: 0.7732 - auc: 0.4523 - precision: 0.0000e+00 - recall: 0.0000e+00 - false_negatives: 1999.0000 - false_positives: 0.0000e+00 - true_negatives: 6813.0000 - true_positives: 0.0000e+00 - val_loss: 0.6476 - val_accuracy: 0.7652 - val_auc: 0.3949 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 887.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 2

INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


In [7]:
%tensorboard --logdir logs/LSTM/fit